In [3]:
!pip install openai

In [ ]:
from openai import OpenAI

client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = "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"
)

completion = client.chat.completions.create(
  model="nvidia/llama-3.3-nemotron-super-49b-v1.5",
  messages=[{"role":"system","content":"/think"}],
  temperature=0.6,
  top_p=0.95,
  max_tokens=65536,
  frequency_penalty=0,
  presence_penalty=0,
  stream=True
)

for chunk in completion:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end="")



In [9]:
import os
import json
import re
import pandas as pd
from openai import OpenAI
from tqdm import tqdm

# =========================
# 설정
# =========================
INPUT_FILE_PATH = "/home/jovyan/work/GitHub/AIFFEL_quest_rs/GoingDeeper/Deeper05/ChatbotData.csv"
OUTPUT_FILE_PATH = "augmented_data_251130.csv"

COL_QUESTION = "Q"
COL_ANSWER   = "A"

client = OpenAI(
    api_key="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",   # export NVIDIA_API_KEY="..."
    base_url="https://integrate.api.nvidia.com/v1"
)

MODEL_ID = "nvidia/llama-3.3-nemotron-super-49b-v1.5"


import re, json
import pandas as pd
from tqdm import tqdm

def generate_augmented_data_stream(question, answer, num_pairs=2):
    prompt = f"""
너는 챗봇 학습 데이터 증강 전문가다.
아래 원본 Q/A로부터 {num_pairs}개의 새로운 Q/A를 생성하라.

[필수 규칙]
1) 출력은 JSON만: {{"pairs":[{{"question":"...","answer":"..."}}, ...]}}
2) question은 원문을 그대로 복사하면 안 된다. (완전 동일/정규화 동일 모두 금지)
3) question은 원문상황에서 이어서 나올 수 있는 질문으로 바꿔라: 구어체/줄임말 다른 말로 표현 등을 섞어라.
4) {num_pairs}개의 question은 서로 겹치면 안 된다.
5) answer도 원문을 그대로 복사하면 안 된다.(완전 동일/정규화 동일 모두 금지)
6) 한국어 이외의 언어는 사용하면 안된다(중국어, 일본어 금지)
7) 질문과 답변은 자연스러운 사람다운 문장이어야 한다
Q: {question}
A: {answer}
""".strip()

    schema = {
        "name": "qa_augmentation",
        "strict": True,
        "schema": {
            "type": "object",
            "additionalProperties": False,
            "properties": {
                "pairs": {
                    "type": "array",
                    "minItems": num_pairs,
                    "items": {
                        "type": "object",
                        "additionalProperties": False,
                        "properties": {
                            "question": {"type": "string"},
                            "answer": {"type": "string"}
                        },
                        "required": ["question", "answer"]
                    }
                }
            },
            "required": ["pairs"]
        }
    }

    completion = client.chat.completions.create(
        model="nvidia/llama-3.3-nemotron-super-49b-v1.5",
        messages=[
            {"role": "system", "content": "Return ONLY JSON that matches the schema."},
            {"role": "user", "content": prompt}
        ],
        response_format={"type": "json_schema", "json_schema": schema},
        temperature=0.7,
        top_p=0.95,
        max_tokens=1200,
        stream=True
    )

    full_text = ""
    for chunk in completion:
        delta = chunk.choices[0].delta
        if delta and delta.content:
            full_text += delta.content

    data = json.loads(full_text)
    return data["pairs"]

def main():
    try:
        df = pd.read_csv(INPUT_FILE_PATH, encoding="utf-8-sig")
    except UnicodeDecodeError:
        df = pd.read_csv(INPUT_FILE_PATH, encoding="cp949")

    if COL_QUESTION not in df.columns or COL_ANSWER not in df.columns:
        raise ValueError(f"CSV에 '{COL_QUESTION}', '{COL_ANSWER}' 컬럼이 없습니다. 현재: {list(df.columns)}")

    augmented_rows = []
    error_count = 0
    empty_pairs_count = 0

    for index, row in tqdm(df.iterrows(), total=len(df)):
        q_text = row[COL_QUESTION]
        a_text = row[COL_ANSWER]

        if pd.isna(q_text) or pd.isna(a_text):
            continue

        q_text = str(q_text)
        a_text = str(a_text)

        augmented_rows.append({COL_QUESTION: q_text, COL_ANSWER: a_text, "type": "original"})

        try:
            pairs = generate_augmented_data_stream(q_text, a_text, num_pairs=3)
        except Exception as e:
            error_count += 1
            pairs = []
            print(f"\n[ERROR] index={index} {e}")

        if not pairs:
            empty_pairs_count += 1

        for pair in pairs:
            new_q = pair.get("question") or pair.get("Question") or pair.get("Q")
            new_a = pair.get("answer")   or pair.get("Answer")   or pair.get("A")
            if new_q and new_a:
                augmented_rows.append({COL_QUESTION: new_q, COL_ANSWER: new_a, "type": "augmented"})

        if (index + 1) % 50 == 0:
            aug_count = sum(1 for r in augmented_rows if r["type"] == "augmented")
            print("\n" + "=" * 90)
            print(f"[{index+1}/{len(df)}] total_rows={len(augmented_rows)}  augmented={aug_count}  empty_pairs={empty_pairs_count}  errors={error_count}")
            print("ORIGINAL Q:", q_text)
            print("ORIGINAL A:", a_text[:200], "...")
            if pairs:
                print("AUG Q:", (pairs[0].get("question") or "")[:200])
                print("AUG A:", (pairs[0].get("answer") or "")[:200], "...")
            else:
                print("(no augmented pairs)")

        if (index + 1) % 200 == 0:
            pd.DataFrame(augmented_rows).to_csv("aug_checkpoint1.csv", index=False, encoding="utf-8-sig")
            print(f"\n[checkpoint saved] aug_checkpoint.csv rows={len(augmented_rows)}")

    out_df = pd.DataFrame(augmented_rows)
    out_df.to_csv(OUTPUT_FILE_PATH, index=False, encoding="utf-8-sig")
    print(f"\n✅ 저장 완료: {OUTPUT_FILE_PATH} (총 {len(out_df)}행)")
    print(f"📌 summary: empty_pairs={empty_pairs_count}, errors={error_count}")

if __name__ == "__main__":
    main()



  0%|          | 50/11823 [01:46<5:47:29,  1.77s/it]


[50/11823] total_rows=200  augmented=150  empty_pairs=0  errors=0
ORIGINAL Q: 감 말랭이 먹고 싶다.
ORIGINAL A: 맛있게 드세요. ...
AUG Q: 감 말랭이 먹고 싶은데 어때요?
AUG A: 좋은 선택이에요. 맛있게 드세요! ...


  1%|          | 100/11823 [03:29<5:37:01,  1.72s/it]


[100/11823] total_rows=400  augmented=300  empty_pairs=0  errors=0
ORIGINAL Q: 거지 같이 일해 놓고 갔어
ORIGINAL A: 일 못하는 사람이 있으면 옆에 있는 사람이 더 힘들죠. ...
AUG Q: 그 사람이 일 잘못했는데도 그냥 갔다는데요?
AUG A: 그럴 때는 옆에서 도와줘야 하는데, 오히려 더 피곤해질 수 있어요. ...


  1%|▏         | 150/11823 [05:13<5:58:21,  1.84s/it]

...


[11750/11823] total_rows=46973  augmented=35223  empty_pairs=9  errors=9
ORIGINAL Q: 패턴 궁금해
ORIGINAL A: 직접 물어보세요. ...
AUG Q: 패턴에 대해 궁금한 점이 있나요?
AUG A: 직접 질문해 주세요. ...


100%|█████████▉| 11800/11823 [7:11:03<01:06,  2.87s/it]


[11800/11823] total_rows=47173  augmented=35373  empty_pairs=9  errors=9
ORIGINAL Q: 혼자 좋아하는 것 같아
ORIGINAL A: 외로운 상태인 거 같아요. ...
AUG Q: 혼자 있는 게 더 좋아하는 것 같아. 왜 그런 걸까?
AUG A: 그렇게 느껴질 수 있는 이유가 다양해요. 예를 들어, 혼자 있는 시간이 필요하거나, 외부 자극에서 벗어나고 싶을 수 있어요. ...

[checkpoint saved] aug_checkpoint.csv rows=47173


100%|██████████| 11823/11823 [7:11:59<00:00,  2.19s/it]


✅ 저장 완료: augmented_data_251130.csv (총 47265행)
📌 summary: empty_pairs=9, errors=9
